**Задача 1: Система управления библиотекой**

Создайте систему управления библиотекой используя collections:

1. Создайте namedtuple `Book` с полями: title, author, isbn, year
2. Создайте namedtuple `Reader` с полями: name, reader_id, phone
3. Используйте `defaultdict(list)` для хранения книг по жанрам
4. Используйте `deque` для очереди читателей, ожидающих популярную книгу
5. Используйте `Counter` для подсчета количества книг каждого автора
6. Используйте `OrderedDict` для хранения истории выдачи книг (читатель -> список книг)
7. Сериализуйте все данные в JSON и pickle форматы

In [11]:
from collections import namedtuple, defaultdict, deque, Counter, OrderedDict
import json
import pickle
from typing import Dict, List, Deque, Any

Book = namedtuple('Book', ['title', 'author', 'isbn', 'year'])
Reader = namedtuple('Reader', ['name', 'reader_id', 'phone'])

class LibrarySystem:
    """Система управления библиотекой"""
    
    def __init__(self) -> None:
        self.books_by_genre: Dict[str, List[Book]] = defaultdict(list)
        self.waiting_queue: Deque[Reader] = deque()
        self.author_counter: Counter = Counter()
        self.history: OrderedDict[Reader, List[Book]] = OrderedDict()

    def add_book(self, book: Book, genre: str) -> None:
        """Добавляет книгу в систему по указанному жанру"""
        self.books_by_genre[genre].append(book)
        self.author_counter[book.author] += 1

    def add_reader_to_queue(self, reader: Reader) -> None:
        """Добавляет читателя в очередь ожидания"""
        self.waiting_queue.append(reader)

    def lend_book(self, reader: Reader, book: Book) -> None:
        """Регистрирует выдачу книги читателю"""
        if reader not in self.history:
            self.history[reader] = []
        self.history[reader].append(book)

    def to_dict(self) -> Dict[str, Any]:
        """Преобразует все данные в словари для сериализации"""
        books_dict = {}
        for genre, books in self.books_by_genre.items():
            books_dict[genre] = [book._asdict() for book in books]

        queue_list = [reader._asdict() for reader in self.waiting_queue]

        counter_dict = dict(self.author_counter)
        

        history_dict = {}
        for reader, books in self.history.items():
            history_dict[reader.reader_id] = {
                'reader': reader._asdict(),
                'books': [book._asdict() for book in books]
            }
        
        return {
            'books_by_genre': books_dict,
            'waiting_queue': queue_list,
            'author_counter': counter_dict,
            'history': history_dict
        }

    def save_json(self, filename: str) -> None:
        """Сохраняет все данные в JSON файл"""
        data = self.to_dict()
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4, ensure_ascii=False)

    def save_pickle(self, filename: str) -> None:
        """Сохраняет всю систему в pickle файл"""
        with open(filename, 'wb') as f:
            pickle.dump(self, f)

    @classmethod
    def load_pickle(cls, filename: str) -> 'LibrarySystem':
        """Загружает систему из pickle файла"""
        with open(filename, 'rb') as f:
            return pickle.load(f)

if __name__ == "__main__":
    library = LibrarySystem()

    book1 = Book("1984", "Джордж Оруэлл", "123-456", 1949)
    book2 = Book("О дивный новый мир", "Олдос Хаксли", "789-012", 1932)
    book3 = Book("Мастер и Маргарита", "Михаил Булгаков", "345-678", 1967)

    library.add_book(book1, "Антиутопия")
    library.add_book(book2, "Антиутопия")
    library.add_book(book3, "Роман")

    reader1 = Reader("Иван Иванов", "001", "+79161234567")
    reader2 = Reader("Петр Петров", "002", "+79167654321")

    library.add_reader_to_queue(reader1)
    library.add_reader_to_queue(reader2)

    library.lend_book(reader1, book1)
    library.lend_book(reader2, book2)
    library.lend_book(reader1, book3)

    library.save_json("library_data.json")
    library.save_pickle("library_data.pkl")

    print("Книги по жанрам:")
    for genre, books in library.books_by_genre.items():
        print(f"{genre}: {len(books)} книг(и)")

    print("\nОчередь читателей:")
    for i, reader in enumerate(library.waiting_queue, 1):
        print(f"{i}. {reader.name}")

    print("\nСтатистика авторов:")
    for author, count in library.author_counter.items():
        print(f"{author}: {count} книг(и)")

    print("\nИстория выдачи:")
    for reader, books in library.history.items():
        print(f"{reader.name}: {len(books)} книг(и)")

Книги по жанрам:
Антиутопия: 2 книг(и)
Роман: 1 книг(и)

Очередь читателей:
1. Иван Иванов
2. Петр Петров

Статистика авторов:
Джордж Оруэлл: 1 книг(и)
Олдос Хаксли: 1 книг(и)
Михаил Булгаков: 1 книг(и)

История выдачи:
Иван Иванов: 2 книг(и)
Петр Петров: 1 книг(и)


In [ ]:
#Реализуйте простую модель стоянки. 
# 1) Скажем, ваши автомобили это именованные кортежи, с указанием марки автомобиля, модели, гос.номера и цвета
# 2) Сама стоянка это двусторонняя очередь (deque)
#   Представим, что у нас нет коллизий, и если места на стоянке нет, то виртуально вызывается эвакуатор и убирает самый крайний автомобиль (как и работает deque)
# 3) Создайте набор автомобилей и в случайном порядке добавляйте/удаляйте их с автостоянки.
# 4) Посчитайте, сколько и каких автомобилей осталось на автостоянке.


In [1]:
from collections import deque, namedtuple
import random
from typing import List, Dict

# 1) Создаем именованный кортеж для автомобилей
Car = namedtuple('Car', ['brand', 'model', 'license_plate', 'color'])

class ParkingLot:
    """Класс автостоянки с ограниченным количеством мест"""
    
    def __init__(self, capacity: int = 5):
        self.capacity = capacity
        self.parking = deque(maxlen=capacity)  # 2) Двусторонняя очередь с ограничением
        self.removed_cars = []  # Для учета эвакуированных автомобилей
    
    def park_car(self, car: Car) -> str:
        """Добавить автомобиль на стоянку"""
        if len(self.parking) < self.capacity:
            self.parking.append(car)
            return f"{car.brand} {car.model} припаркован на свободное место"
        else:
            # 2) Если нет мест - эвакуируем самый старый автомобиль
            removed_car = self.parking.popleft()
            self.removed_cars.append(removed_car)
            self.parking.append(car)
            return f"Эвакуирован {removed_car.brand} {removed_car.model}, припаркован {car.brand} {car.model}"
    
    def remove_car(self, license_plate: str) -> str:
        """Удалить автомобиль по номеру"""
        for i, car in enumerate(self.parking):
            if car.license_plate == license_plate:
                self.parking.remove(car)
                return f"{car.brand} {car.model} уехал со стоянки"
        return f"Автомобиль с номером {license_plate} не найден на стоянке"
    
    def get_parking_stats(self) -> Dict[str, int]:
        """Статистика по автомобилям на стоянке"""
        stats = {}
        for car in self.parking:
            brand = car.brand
            stats[brand] = stats.get(brand, 0) + 1
        return stats
    
    def display_parking(self) -> str:
        """Визуальное отображение стоянки"""
        if not self.parking:
            return " Стоянка пуста"
        
        result = " Текущее состояние стоянки:\n"
        for i, car in enumerate(self.parking, 1):
            result += f"{i}. {car.color} {car.brand} {car.model} [{car.license_plate}]\n"
        return result
    
    def get_evacuated_stats(self) -> Dict[str, int]:
        """Статистика по эвакуированным автомобилям"""
        stats = {}
        for car in self.removed_cars:
            brand = car.brand
            stats[brand] = stats.get(brand, 0) + 1
        return stats

# 3) Создаем набор автомобилей
def generate_cars(count: int = 20) -> List[Car]:
    """Генерация случайных автомобилей"""
    brands_models = {
        'Toyota': ['Camry', 'Corolla', 'RAV4', 'Prius'],
        'Honda': ['Civic', 'Accord', 'CR-V', 'Pilot'],
        'BMW': ['X5', '3 Series', '5 Series', 'X3'],
        'Mercedes': ['C-Class', 'E-Class', 'GLC', 'S-Class'],
        'Audi': ['A4', 'A6', 'Q5', 'Q7'],
        'Hyundai': ['Solaris', 'Elantra', 'Tucson', 'Santa Fe'],
        'Kia': ['Rio', 'Optima', 'Sportage', 'Sorento']
    }
    
    colors = ['Красный', 'Синий', 'Зеленый', 'Черный', 'Белый', 'Серый', 'Желтый']
    
    cars = []
    for i in range(count):
        brand = random.choice(list(brands_models.keys()))
        model = random.choice(brands_models[brand])
        license_plate = f"{random.choice(['A','B','C'])}{random.randint(100, 999)}{random.choice(['AB','CD','EF'])}"
        color = random.choice(colors)
        
        cars.append(Car(brand, model, license_plate, color))
    
    return cars

def main():
    """Основная функция демонстрации работы стоянки"""
    print("=== МОДЕЛЬ АВТОСТОЯНКИ ===\n")
    
    # Создаем стоянку на 5 мест
    parking = ParkingLot(capacity=5)
    
    # Генерируем 15 автомобилей
    cars = generate_cars(15)
    
    print("Сгенерированные автомобили:")
    for i, car in enumerate(cars, 1):
        print(f"{i}. {car.color} {car.brand} {car.model} [{car.license_plate}]")
    
    print(f"\n Вместимость стоянки: {parking.capacity} мест\n")
    
    # 3) Случайные операции с автомобилями
    operations = []
    for i in range(20):
        if random.random() < 0.6 or len(parking.parking) == 0:  # 60% шанс на парковку
            if cars:  # Если есть автомобили для парковки
                car = random.choice(cars)
                cars.remove(car)
                result = parking.park_car(car)
                operations.append(f"Попытка парковки: {result}")
        else:  # 40% шанс на выезд
            if parking.parking:  # Если есть автомобили на стоянке
                car_to_remove = random.choice(list(parking.parking))
                result = parking.remove_car(car_to_remove.license_plate)
                cars.append(car_to_remove)  # Возвращаем автомобиль в общий список
                operations.append(f"Попытка выезда: {result}")
    
    # Выводим историю операций
    print("История операций:")
    for i, op in enumerate(operations[-10:], 1):  # Показываем последние 10 операций
        print(f"{i}. {op}")
    
    print("\n" + "="*50)
    
    # 4) Финальная статистика
    print(parking.display_parking())
    
    # Статистика по оставшимся автомобилям
    current_stats = parking.get_parking_stats()
    print("\n Статистика автомобилей на стоянке:")
    for brand, count in current_stats.items():
        print(f"   {brand}: {count} автомобилей")
    
    # Статистика по эвакуированным автомобилям
    evacuated_stats = parking.get_evacuated_stats()
    if evacuated_stats:
        print("\n Статистика эвакуированных автомобилей:")
        for brand, count in evacuated_stats.items():
            print(f"   {brand}: {count} автомобилей")
        print(f"   Всего эвакуировано: {len(parking.removed_cars)} автомобилей")
    else:
        print("\n Ни один автомобиль не был эвакуирован")

if __name__ == "__main__":
    main()

=== МОДЕЛЬ АВТОСТОЯНКИ ===

Сгенерированные автомобили:
1. Зеленый Kia Sorento [B314EF]
2. Желтый Toyota Prius [A796CD]
3. Серый Hyundai Santa Fe [A324EF]
4. Зеленый Honda Pilot [A789AB]
5. Серый BMW X3 [A180AB]
6. Зеленый BMW X5 [C752CD]
7. Красный BMW X5 [A263AB]
8. Зеленый Toyota Corolla [C412CD]
9. Желтый Honda CR-V [B828CD]
10. Черный Toyota Camry [B113EF]
11. Желтый Hyundai Santa Fe [A185EF]
12. Красный Kia Sorento [B920AB]
13. Белый Kia Rio [B515EF]
14. Серый Toyota RAV4 [C215EF]
15. Черный Audi A6 [C365CD]

 Вместимость стоянки: 5 мест

История операций:
1. Попытка выезда: Honda Pilot уехал со стоянки
2. Попытка парковки: Honda Pilot припаркован на свободное место
3. Попытка парковки: Kia Sorento припаркован на свободное место
4. Попытка выезда: BMW X3 уехал со стоянки
5. Попытка парковки: BMW X3 припаркован на свободное место
6. Попытка парковки: Toyota Prius припаркован на свободное место
7. Попытка парковки: BMW X5 припаркован на свободное место
8. Попытка выезда: Toyota Pri

In [ ]:
# Теперь вы работаете в роли архивариуса РСФСР и занимаетесь хранением информации об иуществе раскулаченных граждан Имперской России
# У вас несколько книг с записями, упорядоченных по алфавиту. Каждая книга хранит фамилии, начинающиеся с конкретной буквы.
# Внутри книги указаны фамилии, к каждому гражданину указана категория имущества, например, мебель, посуда, картины и т.д.
# В каждой категории указан список вещей, которые были изъяты. 
# Вы вносите записи, и если новой записи нет, вы ее создаете. Затем, когда ищите конкретную категорию предмета, начинаете перебирать все книги, начиная с первой буквы по алфавиту в вашем списке.
# Очевидно, что вам нужно использовать defaultdict и ChainMap

In [2]:
from collections import defaultdict, ChainMap
from typing import Dict, List, Any

class ImperialPropertyArchivist:
    """Архивариус имущества раскулаченных граждан Имперской России"""
    
    def __init__(self):
        # Создаем книги для каждой буквы алфавита
        self.books = {}
        self.categories = ['мебель', 'посуда', 'картины', 'драгоценности', 'земля', 'скот', 'инструменты']
        
        # Инициализируем книги для всех букв русского алфавита
        for letter in 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ':
            self.books[letter] = defaultdict(lambda: defaultdict(list))
    
    def add_record(self, surname: str, category: str, items: List[str]) -> None:
        """Добавление записи об изъятом имуществе"""
        first_letter = surname[0].upper()
        
        if first_letter not in self.books:
            print(f" Нет книги для буквы '{first_letter}'")
            return
        
        if category not in self.categories:
            print(f" Неизвестная категория: '{category}'")
            return
        
        # Добавляем запись
        self.books[first_letter][surname][category].extend(items)
        print(f" Добавлено: {surname} - {category}: {', '.join(items)}")
    
    def find_items_by_category(self, category: str, start_letter: str = 'А') -> Dict[str, List[str]]:
        """Поиск всех предметов в категории, начиная с указанной буквы"""
        if category not in self.categories:
            return {}
        
        # Создаем ChainMap для последовательного поиска по книгам
        books_to_search = []
        
        # Находим порядок поиска от start_letter до конца алфавита
        alphabet = 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'
        start_index = alphabet.index(start_letter.upper())
        
        for letter in alphabet[start_index:]:
            if self.books[letter]:
                books_to_search.append(self.books[letter])
        
        if not books_to_search:
            return {}
        
        # Объединяем книги в ChainMap
        combined_books = ChainMap(*books_to_search)
        
        # Ищем все записи в указанной категории
        result = {}
        for surname, properties in combined_books.items():
            if category in properties:
                result[surname] = properties[category]
        
        return result
    
    def find_citizen_property(self, surname: str) -> Dict[str, List[str]]:
        """Поиск всего имущества конкретного гражданина"""
        first_letter = surname[0].upper()
        
        if first_letter in self.books and surname in self.books[first_letter]:
            return dict(self.books[first_letter][surname])
        
        return {}
    
    def display_book(self, letter: str) -> None:
        """Показать содержимое книги для конкретной буквы"""
        letter = letter.upper()
        
        if letter not in self.books or not self.books[letter]:
            print(f"Книга '{letter}' - пуста")
            return
        
        print(f"\n КНИГА БУКВЫ '{letter}':")
        print("=" * 50)
        
        for surname, properties in sorted(self.books[letter].items()):
            print(f"\n Гражданин: {surname}")
            for category, items in properties.items():
                print(f"   {category.capitalize()}: {', '.join(items)}")
    
    def display_statistics(self) -> None:
        """Статистика по архиву"""
        total_citizens = 0
        total_items = 0
        category_stats = defaultdict(int)
        
        for letter, book in self.books.items():
            for surname, properties in book.items():
                total_citizens += 1
                for category, items in properties.items():
                    category_stats[category] += len(items)
                    total_items += len(items)
        
        print("\n СТАТИСТИКА АРХИВА РСФСР:")
        print("=" * 40)
        print(f" Количество книг: {len([l for l in self.books if self.books[l]])}")
        print(f" Учетных граждан: {total_citizens}")
        print(f" Всего изъятых предметов: {total_items}")
        print("\n Распределение по категориям:")
        for category, count in sorted(category_stats.items(), key=lambda x: x[1], reverse=True):
            print(f"   {category.capitalize()}: {count} предметов")

# Демонстрация работы архивариуса
def main():
    # Создаем архивариуса
    archivist = ImperialPropertyArchivist()
    
    print("=== АРХИВ РСФСР - УЧЕТ ИМУЩЕСТВА РАСКУЛАЧЕННЫХ ГРАЖДАН ===\n")
    
    # Добавляем записи об изъятом имуществе
    print("ВНЕСЕНИЕ ЗАПИСЕЙ:")
    
    # Книга А
    archivist.add_record("Антонов", "мебель", ["дубовый стол", "кресло красное", "буфет резной"])
    archivist.add_record("Антонов", "посуда", ["серебряные ложки", "фарфоровый сервиз"])
    archivist.add_record("Алексеев", "земля", ["10 гектаров пашни", "5 гектаров леса"])
    archivist.add_record("Алексеев", "скот", ["3 лошади", "5 коров", "10 овец"])
    
    # Книга Б
    archivist.add_record("Борисов", "картины", ["портрет императора", "пейзаж лесной"])
    archivist.add_record("Белов", "драгоценности", ["золотые часы", "серебряные серьги"])
    archivist.add_record("Белов", "инструменты", ["токарный станок", "набор плотницких инструментов"])
    
    # Книга В
    archivist.add_record("Васильев", "мебель", ["кровать дубовая", "шкаф книжный"])
    archivist.add_record("Волков", "посуда", ["медный самовар", "чайный набор"])
    archivist.add_record("Воробьев", "скот", ["2 лошади", "3 коровы"])
    
    # Книга С
    archivist.add_record("Сидоров", "земля", ["15 гектаров", "мельница водяная"])
    archivist.add_record("Смирнов", "драгоценности", ["бриллиантовое колье", "золотые монеты"])
    
    print("\n" + "="*60)
    
    # Показываем содержимое книг
    archivist.display_book('А')
    archivist.display_book('Б')
    archivist.display_book('В')
    
    print("\n" + "="*60)
    
    # Поиск по категориям (начиная с буквы А)
    print("\n ПОИСК ПРЕДМЕТОВ В КАТЕГОРИИ 'МЕБЕЛЬ' (с буквы А):")
    furniture_items = archivist.find_items_by_category('мебель', 'А')
    for surname, items in furniture_items.items():
        print(f"   {surname}: {', '.join(items)}")
    
    print("\n ПОИСК ПРЕДМЕТОВ В КАТЕГОРИИ 'СКОТ' (с буквы Б):")
    livestock_items = archivist.find_items_by_category('скот', 'Б')
    for surname, items in livestock_items.items():
        print(f"   {surname}: {', '.join(items)}")
    
    print("\n" + "="*60)
    
    # Поиск имущества конкретного гражданина
    print("\n ИМУЩЕСТВО ГРАЖДАНИНА АЛЕКСЕЕВ:")
    alexeev_property = archivist.find_citizen_property("Алексеев")
    for category, items in alexeev_property.items():
        print(f"   {category.capitalize()}: {', '.join(items)}")
    
    print("\n ИМУЩЕСТВО ГРАЖДАНИНА БЕЛОВ:")
    belov_property = archivist.find_citizen_property("Белов")
    for category, items in belov_property.items():
        print(f"   {category.capitalize()}: {', '.join(items)}")
    
    # Статистика архива
    archivist.display_statistics()

if __name__ == "__main__":
    main()

=== АРХИВ РСФСР - УЧЕТ ИМУЩЕСТВА РАСКУЛАЧЕННЫХ ГРАЖДАН ===

ВНЕСЕНИЕ ЗАПИСЕЙ:
 Добавлено: Антонов - мебель: дубовый стол, кресло красное, буфет резной
 Добавлено: Антонов - посуда: серебряные ложки, фарфоровый сервиз
 Добавлено: Алексеев - земля: 10 гектаров пашни, 5 гектаров леса
 Добавлено: Алексеев - скот: 3 лошади, 5 коров, 10 овец
 Добавлено: Борисов - картины: портрет императора, пейзаж лесной
 Добавлено: Белов - драгоценности: золотые часы, серебряные серьги
 Добавлено: Белов - инструменты: токарный станок, набор плотницких инструментов
 Добавлено: Васильев - мебель: кровать дубовая, шкаф книжный
 Добавлено: Волков - посуда: медный самовар, чайный набор
 Добавлено: Воробьев - скот: 2 лошади, 3 коровы
 Добавлено: Сидоров - земля: 15 гектаров, мельница водяная
 Добавлено: Смирнов - драгоценности: бриллиантовое колье, золотые монеты


 КНИГА БУКВЫ 'А':

 Гражданин: Алексеев
   Земля: 10 гектаров пашни, 5 гектаров леса
   Скот: 3 лошади, 5 коров, 10 овец

 Гражданин: Антонов
   Мебе

In [ ]:
# Используя sys и os 
# Выведите некоторые ваши директории с указанием размера всех файлов и упорядочьте их (директории) по времени последнего обращения
# Выведите список расширений файлов, которые хранятся на вашем ПК
# Посчитайте объем памяти, который используется вашим интерпретатором


In [8]:
import os
import sys
import time
from datetime import datetime
from collections import defaultdict, Counter
import gc

def get_directory_size(directory):
    """Рекурсивно вычисляет размер директории"""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            try:
                total_size += os.path.getsize(filepath)
            except (OSError, FileNotFoundError):
                continue
    return total_size

def format_size(size_bytes):
    """Форматирует размер в читаемом виде"""
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size_bytes < 1024.0:
            return f"{size_bytes:.2f} {unit}"
        size_bytes /= 1024.0
    return f"{size_bytes:.2f} PB"

def analyze_directories_by_access_time():
    """Анализирует директории по времени последнего обращения"""
    print("=== АНАЛИЗ ДИРЕКТОРИЙ ПО ВРЕМЕНИ ОБРАЩЕНИЯ ===\n")
    
    # Получаем домашнюю директорию и несколько ключевых путей
    home_dir = os.path.expanduser("~")
    current_dir = os.getcwd()
    
    interesting_dirs = [
        home_dir,
        os.path.join(home_dir, "Desktop"),
        os.path.join(home_dir, "Documents"),
        os.path.join(home_dir, "Downloads"),
        os.path.join(home_dir, "Pictures"),
        current_dir,  # Используем os.getcwd() вместо __file__
    ]
    
    # Добавляем временные директории в зависимости от ОС
    if os.name == 'nt':  # Windows
        temp_dir = os.environ.get('TEMP', '')
        if temp_dir:
            interesting_dirs.append(temp_dir)
    else:  # Linux/Mac
        interesting_dirs.append('/tmp')
    
    # Фильтруем существующие директории
    existing_dirs = [d for d in interesting_dirs if os.path.exists(d)]
    
    # Собираем информацию о директориях
    dir_info = []
    for directory in existing_dirs:
        try:
            stat = os.stat(directory)
            size = get_directory_size(directory)
            access_time = stat.st_atime
            dir_info.append({
                'path': directory,
                'size': size,
                'access_time': access_time,
                'formatted_time': datetime.fromtimestamp(access_time).strftime('%Y-%m-%d %H:%M:%S')
            })
        except (OSError, PermissionError) as e:
            print(f" Ошибка доступа к {directory}: {e}")
            continue
    
    # Сортируем по времени последнего обращения (сначала самые новые)
    dir_info.sort(key=lambda x: x['access_time'], reverse=True)
    
    # Выводим результаты
    print(" ДИРЕКТОРИИ (отсортированы по времени последнего обращения):")
    print("-" * 80)
    for i, info in enumerate(dir_info, 1):
        print(f"{i:2d}. {info['path']}")
        print(f"     Размер: {format_size(info['size'])}")
        print(f"    Последнее обращение: {info['formatted_time']}")
        print()
    
    return dir_info

def analyze_file_extensions():
    """Анализирует расширения файлов в домашней директории"""
    print("\n=== АНАЛИЗ РАСШИРЕНИЙ ФАЙЛОВ ===\n")
    
    home_dir = os.path.expanduser("~")
    extensions = Counter()
    total_files = 0
    
    print(f" Сканируем файлы в {home_dir}...")
    print("Это может занять некоторое время...")
    
    # Ограничиваем глубину сканирования для скорости
    scan_limit = 10000
    
    for dirpath, dirnames, filenames in os.walk(home_dir):
        # Пропускаем системные директории для ускорения
        skip_dirs = ['.git', '__pycache__', '.cache', 'node_modules', '.vscode']
        if any(skip_dir in dirpath for skip_dir in skip_dirs):
            continue
            
        for filename in filenames:
            total_files += 1
            _, ext = os.path.splitext(filename)
            ext = ext.lower() if ext else 'без расширения'
            extensions[ext] += 1
            
            if total_files >= scan_limit:
                break
        
        if total_files >= scan_limit:
            print(f" Достигнут лимит сканирования ({scan_limit} файлов)")
            break
    
    print(f" Проанализировано файлов: {total_files}")
    print(f" Уникальных расширений: {len(extensions)}\n")
    
    # Выводим топ-20 самых распространенных расширений
    print(" ТОП-20 РАСШИРЕНИЙ ФАЙЛОВ:")
    print("-" * 40)
    for ext, count in extensions.most_common(20):
        percentage = (count / total_files) * 100
        print(f"{ext:15} {count:6} файлов ({percentage:.1f}%)")
    
    return extensions, total_files

def analyze_interpreter_memory():
    """Анализирует использование памяти интерпретатором Python"""
    print("\n=== ИСПОЛЬЗОВАНИЕ ПАМЯТИ ИНТЕРПРЕТАТОРОМ ===\n")
    
    # Способ 1: Используем sys для базовой информации
    print(" СИСТЕМНАЯ ИНФОРМАЦИЯ (через sys):")
    print(f"    Версия Python: {sys.version.split()[0]}")
    print(f"    Платформа: {sys.platform}")
    print(f"    Путь к интерпретатору: {sys.executable}")
    print(f"    Кодировка файловой системы: {sys.getfilesystemencoding()}")
    
    # Информация о ссылках на объекты
    print(f"    Количество объектов в памяти: {len(gc.get_objects())}")
    
    # Способ 2: Попробуем использовать psutil, если установлен
    try:
        import psutil
        process = psutil.Process(os.getpid())
        memory_info = process.memory_info()
        
        print("\n ИНФОРМАЦИЯ О ПАМЯТИ (через psutil):")
        print(f"    RSS (Resident Set Size): {format_size(memory_info.rss)}")
        print(f"    VMS (Virtual Memory Size): {format_size(memory_info.vms)}")
        print(f"    Используется Python процессом: {format_size(memory_info.rss)}")
        
    except ImportError:
        print("\n💡 Для детальной информации установите psutil: pip install psutil")

def get_system_info():
    """Выводит общую информацию о системе"""
    print("=== СИСТЕМНАЯ ИНФОРМАЦИЯ ===\n")
    
    print(" ОБЩАЯ ИНФОРМАЦИЯ:")
    print(f"    Текущая рабочая директория: {os.getcwd()}")
    print(f"    Домашняя директория: {os.path.expanduser('~')}")
    print(f"    Имя пользователя: {os.getenv('USER', os.getenv('USERNAME', 'Неизвестно'))}")
    print(f"    Операционная система: {os.name}")
    
    # Информация о дисках (для Unix-систем)
    if hasattr(os, 'statvfs'):
        try:
            statvfs = os.statvfs('/')
            total_space = statvfs.f_frsize * statvfs.f_blocks
            free_space = statvfs.f_frsize * statvfs.f_bavail
            print(f"    Всего места на диске: {format_size(total_space)}")
            print(f"    Свободно места: {format_size(free_space)}")
        except OSError:
            pass

def main():
    """Основная функция анализа"""
    print(" КОМПЛЕКСНЫЙ АНАЛИЗ СИСТЕМЫ")
    print("=" * 60)
    
    # Выводим системную информацию
    get_system_info()
    
    # Анализируем директории
    dir_info = analyze_directories_by_access_time()
    
    # Анализируем расширения файлов
    extensions, total_files = analyze_file_extensions()
    
    # Анализируем использование памяти
    analyze_interpreter_memory()
    
    # Сводная статистика
    print("\n" + "=" * 60)
    print("СВОДНАЯ СТАТИСТИКА:")
    print(f"    Проанализировано директорий: {len(dir_info)}")
    print(f"    Проанализировано файлов: {total_files}")
    print(f"    Уникальных расширений: {len(extensions)}")
    
    if dir_info:
        total_size = sum(d['size'] for d in dir_info)
        print(f"    Общий размер проанализированных директорий: {format_size(total_size)}")

# Запускаем анализ
if __name__ == "__main__":
    main()

 КОМПЛЕКСНЫЙ АНАЛИЗ СИСТЕМЫ
=== СИСТЕМНАЯ ИНФОРМАЦИЯ ===

 ОБЩАЯ ИНФОРМАЦИЯ:
    Текущая рабочая директория: c:\University Sirius_Lizaveta\Python_curse_1semestr\Python_course\lab_1
    Домашняя директория: C:\Users\Елизавета
    Имя пользователя: Елизавета
    Операционная система: nt
=== АНАЛИЗ ДИРЕКТОРИЙ ПО ВРЕМЕНИ ОБРАЩЕНИЯ ===

 ДИРЕКТОРИИ (отсортированы по времени последнего обращения):
--------------------------------------------------------------------------------
 1. C:\Users\9875~1\AppData\Local\Temp
     Размер: 18.96 GB
    Последнее обращение: 2025-09-28 21:08:11

 2. C:\Users\Елизавета\Downloads
     Размер: 10.44 GB
    Последнее обращение: 2025-09-27 12:52:57

 3. C:\Users\Елизавета
     Размер: 290.51 GB
    Последнее обращение: 2025-09-25 15:03:24

 4. c:\University Sirius_Lizaveta\Python_curse_1semestr\Python_course\lab_1
     Размер: 129.57 KB
    Последнее обращение: 2025-09-23 15:33:04

 5. C:\Users\Елизавета\Desktop
     Размер: 160.41 GB
    Последнее обращение: 

**Задача 2: Анализатор файловой системы**

Создайте анализатор файловой системы используя os и sys:

1. Создайте namedtuple `FileInfo` с полями: name, size, extension, modified_time
2. Используйте `os.walk()` для обхода директории
3. Используйте `os.path` функции для получения информации о файлах
4. Используйте `Counter` для подсчета файлов по расширениям
5. Используйте `defaultdict(list)` для группировки файлов по размеру (маленькие < 1MB, средние 1-100MB, большие > 100MB)
6. Используйте `deque` для хранения последних 10 найденных файлов
7. Выведите статистику используя `sys.getsizeof()` для подсчета памяти
8. Сохраните результаты в JSON файл


**Задача 3: Система конфигурации приложения**

Создайте систему конфигурации используя ChainMap и defaultdict:

1. Создайте namedtuple `Config` с полями: key, value, section, default_value
2. Создайте несколько словарей конфигурации (default, user, environment)
3. Используйте `ChainMap` для объединения конфигураций с приоритетом
4. Используйте `defaultdict(dict)` для группировки настроек по секциям
5. Используйте `OrderedDict` для сохранения порядка загрузки конфигураций
6. Используйте `os.environ` для чтения переменных окружения
7. Сериализуйте конфигурацию в JSON и pickle форматы

**Задача 4: Мониторинг системы**

Создайте систему мониторинга используя sys и os:

1. Создайте namedtuple `SystemInfo` с полями: cpu_count, memory_usage, process_id, user_name
2. Используйте `os.cpu_count()` для получения количества процессоров
3. Используйте `sys.getallocatedblocks()` для мониторинга памяти
4. Используйте `os.getpid()` и `os.getlogin()` для информации о процессе
5. Используйте `deque` для хранения последних 20 измерений
6. Используйте `Counter` для подсчета частоты использования различных функций
7. Используйте `defaultdict(list)` для группировки измерений по времени
8. Сохраните историю мониторинга в pickle файл

**Задача 5: Система логирования**

Создайте систему логирования используя все изученные коллекции:

1. Создайте namedtuple `LogEntry` с полями: timestamp, level, message, module, function
2. Используйте `deque` для хранения последних 100 логов (кольцевой буфер)
3. Используйте `defaultdict(list)` для группировки логов по уровням (DEBUG, INFO, WARNING, ERROR)
4. Используйте `Counter` для подсчета количества логов каждого уровня
5. Используйте `OrderedDict` для хранения логов по времени (FIFO)
6. Используйте `ChainMap` для объединения различных источников логов
7. Используйте `os.path` для работы с файлами логов
8. Сериализуйте логи в JSON и pickle форматы

**Задача 6: Кэш-система**

Создайте простую кэш-систему используя collections:

1. Создайте namedtuple `CacheEntry` с полями: key, value, timestamp, access_count
2. Используйте `OrderedDict` для реализации LRU (Least Recently Used) кэша
3. Используйте `deque` для хранения истории доступа к ключам
4. Используйте `Counter` для подсчета частоты доступа к каждому ключу
5. Используйте `defaultdict(int)` для хранения счетчиков доступа
6. Реализуйте методы: get, set, delete, clear, size
7. Используйте `sys.getsizeof()` для мониторинга размера кэша
8. Сериализуйте кэш в pickle формат для сохранения между сессиями


**Задача 7: Анализатор текста**

Создайте анализатор текста используя collections:

1. Создайте namedtuple `WordInfo` с полями: word, frequency, length, first_occurrence
2. Используйте `Counter` для подсчета частоты слов
3. Используйте `defaultdict(list)` для группировки слов по длине
4. Используйте `deque` для хранения последних 50 уникальных слов
5. Используйте `OrderedDict` для хранения слов в порядке первого появления
6. Используйте `os.path` для работы с текстовыми файлами
7. Используйте `sys.getsizeof()` для анализа памяти
8. Сохраните результаты анализа в JSON файл